In [43]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [44]:
import numpy as np
%matplotlib widget
import matplotlib as mpl
import matplotlib.pyplot as plt
import scipy.linalg as la
import pyvista as pv
import imageio.v3 as iio
np.set_printoptions(formatter={"float": "{:10.2f}".format})

In [45]:
from src.experiment import Experiment
from src.pycv.dmcp import dm_to_world
mesh_path = "/home/julian/uni/ushichka-registration/data/formatted/lidar_roi.ply"
mesh_path = "C:/Users/Julian/Nextcloud/Uni/Semester 8/ushichka-registration/data/formatted/lidar_roi.ply"
pvMesh = pv.read(mesh_path)

In [46]:
day = "/home/julian/data/dmcp_experiments/round_05/2018-08-17"
day = "/data/dmcp_experiments/round_05/2018-08-17"
def path_day_cam(day,cam):
    exp_path = f"{day}--cam{cam}"
    return exp_path

In [47]:
pc0 = path_day_cam(day,0)
pc1 = path_day_cam(day,1)
pc2 = path_day_cam(day,2)

exp0 = Experiment(pc0,mesh_path)
exp1 = Experiment(pc1,mesh_path)
exp2 = Experiment(pc2,mesh_path)
exps = [exp0, exp1, exp2]

In [48]:
from src.pyushichka import loadCalibration
ps = [loadCalibration(0,r"C:\data\ushichka\2018-08-17"),loadCalibration(1,r"C:\data\ushichka\2018-08-17"),loadCalibration(2,r"C:\data\ushichka\2018-08-17")]
pts = []
for cam in ps:
    K, P = cam
    #P_trans = P @ la.inv(trans)
    pos = (la.null_space(P) / la.null_space(P)[-1])[:3].flatten()
    pts.append(pos)

pts = np.array(pts)
pts_hat = np.hstack((pts,np.ones((pts.shape[0],1))))
pts_hat

array([[     -2.80,       1.97,      -0.31,       1.00],
       [     -0.93,       4.12,      -2.84,       1.00],
       [      0.64,       4.10,      -1.40,       1.00]])

In [49]:
from dataclasses import dataclass
@dataclass
class TransformedCameras:
    pts: np.ndarray = np.array([])
tcs = []
for exp in exps:
    trans = exp.load_transform()[:3,:]
    tc = TransformedCameras(pts=(trans @ pts_hat.T).T)
    tcs.append(tc)

tcs

[TransformedCameras(pts=array([[      1.86,       2.02,       0.18],
        [      2.11,      -1.18,      -1.86],
        [      0.69,      -1.95,      -0.48]])),
 TransformedCameras(pts=array([[      1.83,       1.95,       0.36],
        [      2.15,      -1.28,      -1.63],
        [      0.69,      -2.04,      -0.27]])),
 TransformedCameras(pts=array([[      1.78,       1.98,       0.32],
        [      2.11,      -1.10,      -1.89],
        [      0.69,      -1.96,      -0.57]]))]

In [50]:
plotter = pv.Plotter(notebook=False)
for i in range(len(tcs)):
    tc = tcs[i]
    sv = pv.PolyData(tc.pts)
    plotter.add_mesh(sv, color=["red","green","blue"][i], point_size=25,render_points_as_spheres=True)
plotter.add_mesh(pvMesh)
plotter.show()